In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime as dt
from scipy import stats
import openpyxl
import calendar

In [2]:
# find difference between 2 dates in months

def month_diff(x,y):
    end = x.dt.to_period('M').view(dtype='int64')
    start = y.dt.to_period('M').view(dtype='int64')
    return end - start

In [4]:
# read transfer order by product Singapore

path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\auto_planner\\'

df = pd.read_excel(path+'Transfer Order by Product6_6_2023 9 38 37 AM.xlsx', header=4)
df.columns = [i.lower() for i in df.columns]
df.columns = df.columns.str.replace(' ','_')
df.rename(columns={'value_in_base_currency':'sales','sales_person_name':'sales_rep',
                   'sales_order_date':'date'}, inplace=True)
df.date = pd.to_datetime(df.date, format='%d-%m-%Y')
df.account = df.account.astype(str)
df.account= [' '.join(i.split()) for i in df.account]
df.product_code = df.product_code.astype(str)
df.product_code=[' '.join(i.split()) for i in df.product_code]

df.rename(columns={'class':'cltv'},inplace=True)

df = df[['date','account_code','account','product_code','product_name','sales_rep','sales','cltv','country']]
df.account = [i.lower() for i in df.account]

sg_to = df[df.country=='Singapore']

## clustering address in Sg account list

In [5]:
# read account info for Singapore

ac = pd.read_excel(path+'sea_account.xlsx')
ac.columns = ac.columns.str.lower()
ac.columns = ac.columns.str.replace(' ','_')
ac.rename(columns={'class':'cltv','sales_executive':'sales_rep','region':'country'}, inplace=True)
ac.postal_code= ac.postal_code.replace(' ',np.nan, regex=True)
ac1=ac[['account_code','account_name','country', 'city','postal_code','billing_address','cltv']]
ac1.country = [i.lower() for i in ac1.country]

sg_ac = ac1[ac1.country=='singapore']


In [6]:
sg_ac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 961 entries, 17700 to 21908
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account_code     961 non-null    object
 1   account_name     961 non-null    object
 2   country          961 non-null    object
 3   city             961 non-null    object
 4   postal_code      504 non-null    object
 5   billing_address  961 non-null    object
 6   cltv             950 non-null    object
dtypes: object(7)
memory usage: 60.1+ KB


In [7]:
# accounts not in transfer order
missing_in_to = set(sg_to.account_code)-set(sg_ac.account_code)

In [8]:
# accounts in both transfer order and account list
sgac_and_sgto = set(sg_to.account_code) & set(sg_ac.account_code)

both = list(sgac_and_sgto)

In [9]:
# update account address list according to accounts common to both transfer order and account list
sgac1=pd.DataFrame()
for i in sg_ac.account_code:
    if i in both:
        x = sg_ac[sg_ac.account_code==i]
        sgac1 = sgac1.append(x)

# reset index of updated account list to include only updated transfer order sales record
sgac1 = sgac1.reset_index(drop=True)

In [10]:
sgac1.head()

,account_code,account_name,country,city,postal_code,billing_address,cltv
0,SEA-014242,@Just Braces Dental Centre,singapore,Singapore,239519,"3 Killiney Road, #09-02, Winsland House 1,Sing...",B
1,SEA-014243,1728 Dental Practice,singapore,Singapore,NaN,"Blk 135 Jurong Gateway Road\n#01-319,Singapore...",B
2,SEA-014247,A Dental Clinic (Clementi),singapore,Singapore,120450,"450 Clementi Ave 3 #01-285,Singapore, Clementi...",B
3,SEA-014248,A Dental Clinic (Jurong East Central),singapore,Singapore,600135,"135 Jurong Gateway Road, #01-333,Singapore, Ju...",B
4,SEA-014257,Accord Dental Clinic,singapore,Singapore,NaN,"442 PASIR RIS DR 6\n #01 - 26,Singapore, Pasir...",B


In [11]:
# extract postal code from billing address

import re

for i, n in zip(sgac1.index, sgac1.billing_address):
    
    # initializing K 
    K = 6
    # using regex() to solve problem
    temp = re.search('\d{% s}'% K, n)
    res = (temp.group(0) if temp else '')
    sgac1.loc[i,'postal_code']=res
    sg = sgac1.copy()
    sg = sg.reset_index(drop=True)

In [12]:
sg.head()

,account_code,account_name,country,city,postal_code,billing_address,cltv
0,SEA-014242,@Just Braces Dental Centre,singapore,Singapore,239519,"3 Killiney Road, #09-02, Winsland House 1,Sing...",B
1,SEA-014243,1728 Dental Practice,singapore,Singapore,600135,"Blk 135 Jurong Gateway Road\n#01-319,Singapore...",B
2,SEA-014247,A Dental Clinic (Clementi),singapore,Singapore,120450,"450 Clementi Ave 3 #01-285,Singapore, Clementi...",B
3,SEA-014248,A Dental Clinic (Jurong East Central),singapore,Singapore,600135,"135 Jurong Gateway Road, #01-333,Singapore, Ju...",B
4,SEA-014257,Accord Dental Clinic,singapore,Singapore,510442,"442 PASIR RIS DR 6\n #01 - 26,Singapore, Pasir...",B


In [13]:
def classify():
    # compute number of days from last purchase
#     _ = df[df.country==cty]
    _ = sg_to.copy()
    _['current_date']=pd.to_datetime('today')
    _['since_last']=month_diff(_.current_date,_.date)
    
    req = (_.since_last >= 6)
    _=_[req]
#     _=_[_.since_last >= 6]
    class_a = _[_.cltv=='A']
    class_b = _[_.cltv=='B']
    class_c = _[_.cltv=='C']
    return class_a, class_b, class_c


## Sg account list to categorise accounts into clusters

In [14]:
# create a new column consisting of first 2 digits of postal code (for clustering into districts)

sg.postal_code = sg.postal_code.astype(str)

# create postal sector to be clustered

sg['postal_sec']=np.nan

for i, p in zip(sg.index,sg.postal_code):
    x = p[0:2]
    sg.loc[i,'postal_sec']=x

In [15]:
# categorizing into districts


s1=['01','02','03','04','05','06']
s2=['07','08']
s3=['14','15','16']
s4=['09','10']
s5=['11','12','13']
s6=['17']
s7=['18','19']
s8=['20','21']
s9=['22','23']
s10=['24','25','26','27']
s11=['28','29','30']
s12=['31','32','33']
s13=['34','35','36','37']
s14=['38','39','40','41']
s15=['42','43','44','45']
s16=['46','47','48']
s17=['49','50','81']
s18=['51','52']
s19=['53','54','55','82']
s20=['56','57']
s21=['58','59']
s22=['60','61','62','63','64']
s23=['65','66','67','68']
s24=['69','70','71']
s25=['72','73']
s26=['77','78']
s27=['75','76']
s28=['79','80']


In [16]:
# generate numbers for district
u=1
sn=[]
while u <=28:
    j='d'+ str(u)
    sn.append(j)
    u=int(u)
    u+=1

In [17]:
# generating running numbers with prefix 's'
s=[s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28]

In [18]:
# classifying postal sectors into districts

sg['cluster']=np.nan

for idx, i in zip(sg.index, sg.postal_sec):
    for d,d1 in zip(s,sn):
        for f in d:
            if i in f:
                sg.loc[idx,'cluster']=d1
        

In [19]:
# extract class A & B & C customers

# cty='Singapore'
a,b,c=classify()

abc1 = pd.concat([a,b,c])

# ab = a.append(b)
# abc = ab.append(c)
# abc1 = ab.append(c)

In [20]:
# preview for sales opportunity

abc1.head()

,date,account_code,account,product_code,product_name,sales_rep,sales,cltv,country,current_date,since_last
562,2022-12-23,SEA-014258,ace dental centre,462520,Tooth Mousse Strawberry,Patrick Teo,208,A,Singapore,2023-06-06 09:43:33.486032,6
577,2022-12-21,SEA-0026860,advanced dental clinic (bukit panjang),0101B677,Fuji II LC Capsule,Patrick Teo,428,A,Singapore,2023-06-06 09:43:33.486032,6
738,2022-11-22,SEA-014579,koh dental surgery west coast,0129F243,G-aenial ANTERIOR,Patrick Teo,243,A,Singapore,2023-06-06 09:43:33.486032,7
739,2022-11-22,SEA-014579,koh dental surgery west coast,0138F282,G-Premio Bond Refill,Patrick Teo,448,A,Singapore,2023-06-06 09:43:33.486032,7
740,2022-11-22,SEA-014579,koh dental surgery west coast,0129F243,G-aenial ANTERIOR,Patrick Teo,486,A,Singapore,2023-06-06 09:43:33.486032,7


In [21]:
# selecting sales person

abc1 = abc1[abc1.sales_rep=='Patrick Teo']

In [22]:
abc1.sales_rep.value_counts()

Patrick Teo    1217
Name: sales_rep, dtype: int64

## CRM sales opportunity

In [23]:
# renaming columns of sales opportunity
abc_1 = abc1[['date','cltv','account_code','account','product_code','product_name','sales','sales_rep',]]
abc_1=abc_1.rename(columns={'date':'Date Of Last Purchase','product_name':'Product Name','sales_rep':'Sales Person Name',
                    'cltv':'Class','account':'Account','sales':'Sales Amount','product_code':'Product Code','account_code':
                         'Account Code'})
# abc_.to_csv('crm_sales_oppy.csv',index=False)

In [24]:
# saving to crm sales opportunity in excel format
# making a copy
xx = abc_1.copy()

# convert data to string format
xx['Date Of Last Purchase'] = [i.strftime("%m/%d/%Y") for i in xx['Date Of Last Purchase']]

# saving sales opportunity to excel format
xx.to_excel(path+'crm_sales_oppy_patrick.xlsx',index=False)

## Find out all the weekdays of the month

In [25]:
# holidays

hol = {12:[25,26], 1:[1,22,23], 2:0, 3:0, 4:7, 5:1, 6:[2,29],7:0, 8:9,11:12,12:25}

In [26]:
# get weekdays
def get_weekdays(mth, yr=2022):
    cal = calendar.monthcalendar(yr,mth)
    tot=[]
    for n in range(len(cal)):
        for i in range(0,5):
            a = cal[n][i]
            if a!=0:
                tot.append(a)
            else:
                continue
    return tot

# remove holidays
def holidays(mth,tot):
    tot2=[]
    h = hol[mth]
    if type(h)==list:
        for i in tot:
            if i not in h:
                tot2.append(i)
    elif type(h)!=0:
        for i in tot:
            if i!=h:
                tot2.append(i)
            else:
                continue
    else:
        return tot
    
    return tot2

In [27]:
# create crm class

class crm():
    def __init__(self,code,name, meeting, month, date, year, stime, etime,dayname, opp, desc, cocaller, cl, sales_rep):
        self.code = code
        self.name = name
        self.meeting = meeting
        self.month = month
        self.date = date
        self.year = year
        self.stime = stime
        self.etime = etime
        self.dayname = dayname
        self.opp = opp
        self.desc = desc
        self.cocaller = cocaller
        self.cl=cl
        self.sales_rep = sales_rep
    
        
    def fdate(self):
        return "{}/{}/{}".format(self.month,self.date,  self.year)

In [28]:
t1="10:00:00 AM"
t2="11:30:00 AM"
t3='1:00:00 PM'
t4="2:30:00 PM"
t5="4:00:00 PM"
t6="5:30:00 PM"
t7="7:00:00 PM"

timelist=[t1,t2,t3,t4,t5,t6,t7]

In [29]:
def dropDup(mylist):
    uniquelist=set()
    for s in mylist:
        uniquelist.add(s)
    newlist = list(uniquelist)
    return newlist

In [30]:
# groupby account code for transfer order
a1 = abc1.groupby(['account_code','account','sales_rep']).agg({'product_name':lambda x: list(x)}).reset_index()

# remove duplicates in product name

a1.product_name = [dropDup(i) for i in a1.product_name if i != np.nan]

In [31]:
sg['salesrep']='Patrick Teo'

In [32]:
sg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account_code     148 non-null    object
 1   account_name     148 non-null    object
 2   country          148 non-null    object
 3   city             148 non-null    object
 4   postal_code      148 non-null    object
 5   billing_address  148 non-null    object
 6   cltv             148 non-null    object
 7   postal_sec       148 non-null    object
 8   cluster          148 non-null    object
 9   salesrep         148 non-null    object
dtypes: object(10)
memory usage: 11.7+ KB


In [33]:
sg.to_csv(path+'patrick_account_detail.csv', index=False)

# attached cluster number to transfer order

In [34]:
# pd.merge(a1,sg,how='left',on='account_code')

In [35]:
# attached cluster number to transfer order

a1['cluster']=np.nan
for i,a in zip(a1.index,a1.account_code):
    for aa, c in zip(sg.account_code, sg.cluster):
        if a==aa:
            a1.loc[i,'cluster']=c

In [36]:
# check the number of rows
a1.sales_rep.value_counts()

Patrick Teo    150
Name: sales_rep, dtype: int64

In [37]:
# remove nan based on column 'cluster'
a2 = a1.dropna(subset=['cluster'])

In [38]:
abc1.head()

,date,account_code,account,product_code,product_name,sales_rep,sales,cltv,country,current_date,since_last
562,2022-12-23,SEA-014258,ace dental centre,462520,Tooth Mousse Strawberry,Patrick Teo,208,A,Singapore,2023-06-06 09:43:33.486032,6
577,2022-12-21,SEA-0026860,advanced dental clinic (bukit panjang),0101B677,Fuji II LC Capsule,Patrick Teo,428,A,Singapore,2023-06-06 09:43:33.486032,6
738,2022-11-22,SEA-014579,koh dental surgery west coast,0129F243,G-aenial ANTERIOR,Patrick Teo,243,A,Singapore,2023-06-06 09:43:33.486032,7
739,2022-11-22,SEA-014579,koh dental surgery west coast,0138F282,G-Premio Bond Refill,Patrick Teo,448,A,Singapore,2023-06-06 09:43:33.486032,7
740,2022-11-22,SEA-014579,koh dental surgery west coast,0129F243,G-aenial ANTERIOR,Patrick Teo,486,A,Singapore,2023-06-06 09:43:33.486032,7


## Iterate over days of the month

In [39]:
# allocate visits to calendar based on districts

def plan1(tt,rep, mth,cl, s=0, yr=2023):
    
    wk = get_weekdays(mth,yr)
    wkdays = holidays(mth,wk)
    
    rep = a2[a2.sales_rep==rep]
    m = rep[rep.cluster==cl].reset_index(drop=True)
    if m.shape[0]<tt: 
        tt=m.shape[0]
    k = m.shape[0]//tt #-----------no. of days needed to fulfill all customer visit tt customer/day a certain district
    
    code=[]
    account=[]
    meeting=[]
    date=[]
    start=[]
    end=[]
    dayname=[]
    oppy_pdt=[]
    description=[]
    cocaller=[]
    city=[]
    rep=[]
    e = s+k
    for n in wkdays[s:e]:
        for i in range(tt): 
            # assigning values to class attributes
            c1=crm(m.loc[i,'account_code'],m.loc[i,'account'], 'telemarketing', mth,n,yr,timelist[i],timelist[i+1],calendar.weekday(2022,mth,n),'Y', m.loc[i,'product_name'],'N',cl, 
                   m.loc[i,'sales_rep'])
            code.append(c1.code)
            account.append(c1.name)
            meeting.append(c1.meeting)
            date.append(c1.fdate())
            start.append(c1.stime)
            end.append(c1.etime)
            dayname.append(c1.dayname)
            oppy_pdt.append(c1.opp)
            description.append(c1.desc)
            cocaller.append(c1.cocaller)
            city.append(c1.cl)
            rep.append(c1.sales_rep)
            
            if i==tt-1: 
                m=m.loc[tt::].reset_index(drop=True) 
            else:
                continue
            
            
    df=pd.DataFrame({'account_code':code,'account':account,'meeting':meeting,'date':date,'start':start,'end':end,'day':dayname,'oppy_pdt?':oppy_pdt,
               'description':description,'Co-caller?':cocaller,'city':cl,'sales_rep':rep})
    
    return df, e


def cluster(rep):
    _ = a2[a2.sales_rep==rep]
    return _.cluster.unique()


def index2int(dff,x):
    i = dff[dff.account==x].index.tolist()
    return int((" ".join(map(str, i))))


def index2str(i):
    return str((" ".join(map(str, i))))

def firstrow(p):
    _=p.iloc[[0]].T
    return _.T

## run solution

In [56]:
rep='Patrick Teo'
tt=5 # number of visits each day
s=0
mth=6
yr=2023
tot=pd.DataFrame()
for i in cluster(rep):
    df, s = plan1(tt, rep,mth,i,s,yr)
    tot=tot.append(df)
tot1=tot.reset_index(drop=True)
tot2=tot1.copy()
tot=tot.reset_index(drop=True)

# removing square brackets from description column
tot.description = [','.join(i) for i in tot.description]

# adjustments to dataframe
del tot['day']
# del tot['city']
tot['Co-caller name']='none'

# trimming columns
# tot=(tot[['account_code','account','meeting','date','start','end','oppy_pdt?', 'description','Co-caller?','Co-caller name',
#          'sales_rep']])

In [57]:
# remove aprostephy from account
tot.account=tot.account.str.replace("'","")

In [58]:
def convertmonth(mth):
    return calendar.month_abbr[mth]

In [59]:
tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   account_code    93 non-null     object
 1   account         93 non-null     object
 2   meeting         93 non-null     object
 3   date            93 non-null     object
 4   start           93 non-null     object
 5   end             93 non-null     object
 6   oppy_pdt?       93 non-null     object
 7   description     93 non-null     object
 8   Co-caller?      93 non-null     object
 9   city            93 non-null     object
 10  sales_rep       93 non-null     object
 11  Co-caller name  93 non-null     object
dtypes: object(12)
memory usage: 8.8+ KB


In [60]:
# merge 2 columns

In [61]:
tot1 = tot.copy()

In [62]:
tot1['startm']=tot1.apply(lambda row: '{} {}'.format(row['date'], row['start']), axis=1)

tot1['endm']=tot1.apply(lambda row: '{} {}'.format(row['date'], row['end']), axis=1)

del tot1['start']
del tot1['end']

In [63]:
tot1.head()

,account_code,account,meeting,date,oppy_pdt?,description,Co-caller?,city,sales_rep,Co-caller name,startm,endm
0,SEA-0026860,advanced dental clinic (bukit panjang),telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,Fuji II LC Capsule",N,d23,Patrick Teo,none,6/1/2023 10:00:00 AM,6/1/2023 11:30:00 AM
1,SEA-014315,bethel dental surgery,telemarketing,6/1/2023,Y,"Fuji Ortho LC Intro Pack 1-1,Fuji IX GP Extra ...",N,d23,Patrick Teo,none,6/1/2023 11:30:00 AM,6/1/2023 1:00:00 PM
2,SEA-014333,bukit batok dental surgery,telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,Fuji Plus Capsule,Fuj...",N,d23,Patrick Teo,none,6/1/2023 1:00:00 PM,6/1/2023 2:30:00 PM
3,SEA-014372,citizen dental surgery,telemarketing,6/1/2023,Y,"everX Flow Bulk,FujiCEM Refill,Solare X 2mL,Fl...",N,d23,Patrick Teo,none,6/1/2023 2:30:00 PM,6/1/2023 4:00:00 PM
4,SEA-014461,eastern dental surgery (bukit batok),telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,G-aenial Universal Fl...",N,d23,Patrick Teo,none,6/1/2023 4:00:00 PM,6/1/2023 5:30:00 PM


In [64]:
# from datetime import datetime
# import pytz

# # Define the source datetime in the original format
# source_datetime_str = "7/13/2023 11:30:00 AM"

# # Create a datetime object from the source datetime string
# source_datetime = datetime.strptime(source_datetime_str, "%m/%d/%Y %I:%M:%S %p")

# # Define the Singapore time zone
# sg_timezone = pytz.timezone("Asia/Singapore")

# # Convert the datetime to Singapore time
# sg_datetime = sg_timezone.localize(source_datetime)

# # Convert the datetime to the ISO 8601 format
# iso8601_datetime = sg_datetime.isoformat()

# print(iso8601_datetime)


In [65]:
# Change datetime to ISO1806 format

import pytz

def timeformat(t):


    # Create a datetime object from the source datetime string
    source_datetime = datetime.strptime(t, "%m/%d/%Y %I:%M:%S %p")

    # Define the Singapore time zone
    sg_timezone = pytz.timezone("Asia/Singapore")

    # Convert the datetime to Singapore time
    sg_datetime = sg_timezone.localize(source_datetime)

    # Convert the datetime to the ISO 8601 format
    iso8601_datetime = sg_datetime.isoformat()
    
    return iso8601_datetime

In [66]:
# convert columns to ISO1806 format

tot1.startm = [timeformat(t) for t in tot1.startm]

In [67]:
tot1.endm = [timeformat(t) for t in tot1.endm]

In [68]:
tot1.head()

,account_code,account,meeting,date,oppy_pdt?,description,Co-caller?,city,sales_rep,Co-caller name,startm,endm
0,SEA-0026860,advanced dental clinic (bukit panjang),telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,Fuji II LC Capsule",N,d23,Patrick Teo,none,2023-06-01T10:00:00+08:00,2023-06-01T11:30:00+08:00
1,SEA-014315,bethel dental surgery,telemarketing,6/1/2023,Y,"Fuji Ortho LC Intro Pack 1-1,Fuji IX GP Extra ...",N,d23,Patrick Teo,none,2023-06-01T11:30:00+08:00,2023-06-01T13:00:00+08:00
2,SEA-014333,bukit batok dental surgery,telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,Fuji Plus Capsule,Fuj...",N,d23,Patrick Teo,none,2023-06-01T13:00:00+08:00,2023-06-01T14:30:00+08:00
3,SEA-014372,citizen dental surgery,telemarketing,6/1/2023,Y,"everX Flow Bulk,FujiCEM Refill,Solare X 2mL,Fl...",N,d23,Patrick Teo,none,2023-06-01T14:30:00+08:00,2023-06-01T16:00:00+08:00
4,SEA-014461,eastern dental surgery (bukit batok),telemarketing,6/1/2023,Y,"Fuji IX GP Extra Capsule,G-aenial Universal Fl...",N,d23,Patrick Teo,none,2023-06-01T16:00:00+08:00,2023-06-01T17:30:00+08:00


In [55]:
# saving file to excel format file

tot1.to_excel(f"patrick_{convertmonth(mth)}.xlsx",index=False)

In [69]:
# saving file to excel format file

tot1.to_excel(f"patrick_Jun.xlsx",index=False)

In [ ]:
tot1

## workaround on start time

In [233]:
# def workaround(x):
#     trans = lambda x: '10:01 AM' if x=='10:00 AM' else '11:31 AM' if x=='11:30 AM' else '01:01 PM' if x=='01:00 PM'\
#     else '02:31 PM' if x=='02:30 PM' else '04:01 PM' if x=='04:00 PM' else '05:31 PM' if x=='05:30 PM' else '07:01 PM' if \
#     x=='07:00 PM' else '07:01 AM' if x=='07:00 AM' else '08:01 AM' if x=='08:00 AM' else '09:01 AM' if x=='09:00 AM' \
#     else 'na'
#     return trans(x)
    

In [234]:
# tot.start = [workaround(x) for x in tot.start]

In [235]:
tot.to_excel('dec_planner_v2.xlsx',index=False)

In [236]:
tot5=tot.set_index('account')

## transform into google calendar format

In [237]:
def csv2google(dfff):
    dfff=dfff.reset_index()
    dfff.rename(columns={'account':'Subject','date':'Start Date','start':'Start Time','end':'End Time'},inplace=True)
    dfff=dfff[['Subject','Start Date','Start Time','End Time']]
    return dfff
    

In [238]:
totgoogle = csv2google(tot)

In [240]:
totgoogle.to_csv(path+'dec_planner_go.csv',index=False)

## bug fixing

In [ ]:
tot5=tot.set_index('account')

## preference allocator

In [ ]:
def crmplanner(pref, df):

    tot5=df.set_index('account')

    # check if district is in the list
    tot2=pd.DataFrame()
    for c in df.city.unique():
        for ct in pref.city.unique():
            if c==ct:
                aa = df[df.city==ct] # extract the dataframe with selected city district
                tot2=tot2.append(aa)

            else:
                continue

    aa = tot2.copy()


    # check if the account relating preferred time slot is in the original dataframe list of accounts
    tot3=[]
    for acc in pref.index.unique():
        for ac in aa.account.unique():
            if acc==ac:
                tot3.append(acc) # returns a list of accounts which tally with accounts with preferred time slot 
                len_list = len(tot3)

            else:
                continue


    for i in tot3:
        # extract customer and preferred T/S (df)
        a_pref = pref.loc[[i]].T
        a_pref = a_pref.T

        # extract name of customer from preferred list
        a_pref_name = index2str(a_pref.index) # a_pref.index is based on iterator 'i'
        a_pref_city = a_pref.city[0]
        a_pref_time = a_pref.pref_time[0]
        a_pref_day = a_pref.pref_day[0]

        # check to see whether a_pref_name, a_pref_city, a_pref_time are satisfied in a single customer
        # extract customer, city and start time from original list

        for s in tot5.index: # iterate over account names in original list
            if a_pref_name == s: # check whether pref account name is found in the original list
                a_orig = tot5.loc[[i]].T
                a_orig = a_orig.T
                a_orig_name = index2str(a_orig.index)
                a_orig_city = a_orig.city[0]
                a_orig_time = a_orig.start[0]
                a_orig_day = a_orig.day[0]



            else:
                continue

                # find out whether all conditions are satisfied for account name, city and preferred time
                # conditions satisfied
            if a_pref_name==a_orig_name and a_pref_city==a_orig_city and a_pref_time==a_orig_time and a_pref_day==a_orig_day:
                print(f"{a_pref_name}: Preferred day and time satisfied")
                print()
                continue # exit if all conditions are satisfied

            # conditions satisfied
            elif a_pref_name==a_orig_name and a_pref_city==a_orig_city and a_pref_time==a_orig_time:
                print(f"{a_pref_name}: Preferred time satisfied")
                print()
                continue # exit if all conditions are satisfied

            elif a_pref_name==a_orig_name and a_pref_city==a_orig_city:


                # extract original df with a_pref_city
                a_orig_df_city = tot5[tot5.city==a_orig_city]

                # check whether preferred day is in original crm list with selected district
                if a_pref_day in a_orig_df_city.day.tolist():


                    # extract df based on preferred day, preferred day available
                    a_pref_day_df = a_orig_df_city[a_orig_df_city.day==a_pref_day]

                    num_of_appoints = a_orig_df_city[a_orig_df_city.day==a_pref_day]


                    # iterate over account name and start time to find a match with preferred time
                    for i, d in zip(a_pref_day_df.index,a_pref_day_df.start):

                        if d==a_pref_time: # preferred day available

                            print()
                            target_acct_with_pref_time = a_pref_day_df.loc[[i]].T
                            target_acct_with_pref_time = target_acct_with_pref_time.T

                            # extract name of target acct with preferred time
                            target_acct_with_pref_time_name = index2str(target_acct_with_pref_time.index)

                            # switch account names between 'target_acct_with_pref_time_name' and 'a_pref_name'
                            b_target_acct_with_pref_time=target_acct_with_pref_time.rename\
                            (index={target_acct_with_pref_time_name:a_pref_name})
                            b_orig = a_orig.rename(index={a_pref_name:target_acct_with_pref_time_name})

                            # delete these 2 names from original crm list "tot5"
                            tot5 = tot5.drop([target_acct_with_pref_time_name,a_pref_name])

                            tot5=tot5.append(b_target_acct_with_pref_time)
                            tot5=tot5.append(b_orig)

                            print(f"{a_pref_name}: Preferred day and time satisfied")
                            print()
                        else:
                            a_orig.loc[a_pref_name,'start']=a_pref_time
                            tot5=tot5.drop(a_pref_name)
                            tot5=tot5.append(a_orig)


                    # when the preferred time is in the original list, just switch place with another account        
                elif a_pref_time in a_orig_df_city.start.tolist():

                    target_acct_with_pref_time = a_orig_df_city[a_orig_df_city.start==a_pref_time]
                    target_acct_with_pref_time = firstrow(target_acct_with_pref_time)

                    # extract name of target acct with preferred time
                    target_acct_with_pref_time_name = index2str(target_acct_with_pref_time.index)

                    # switch account names between 'target_acct_with_pref_time_name' and 'a_pref_name'
                    b_target_acct_with_pref_time=target_acct_with_pref_time.rename\
                    (index={target_acct_with_pref_time_name:a_pref_name})
                    b_orig = a_orig.rename(index={a_pref_name:target_acct_with_pref_time_name})

                    # delete these 2 names from original crm list "tot5"
                    tot5 = tot5.drop([target_acct_with_pref_time_name,a_pref_name])

                    tot5=tot5.append(b_target_acct_with_pref_time)
                    tot5=tot5.append(b_orig)

                    print(f"{a_pref_name}: Preferred time satisfied")
                    print()


                    # when preferred time is not in the original list, just insert the time
                else:
                    a_orig.loc[a_pref_name,'start']=a_pref_time
                    tot5=tot5.drop(a_pref_name)
                    tot5=tot5.append(a_orig)
                    
    return tot5


In [ ]:
pref = pd.DataFrame({'account':['Royce Dental Surgery (Bidadari)','CITIZEN DENTAL SURGERY','Implant Dental Centre'], 
                     'pref_time': [t1,t6,t5],'pref_day':[4,1,3],'city':['d13','d23','d10']})

# pref = pd.DataFrame({'account':['Tooth Matters Dental Surgery (Toa Payoh)'], 
#                      'pref_time': [t3],'pref_day':[2],'city':['d12']}, index=[0])

pref = pref.set_index('account')

In [ ]:
# temp=temp.reset_index()

In [ ]:
# temp.to_csv('crm_calendar_v1.csv', index=False)

In [ ]:
pref.reset_index()
pref.to_csv('pref.csv', index=False)

In [ ]:
rep='Patrick Teo'
tt=6
s=0
mth=7
tot=pd.DataFrame()
for i in cluster(rep):
    df, s = plan1(tt, rep,mth,i,s)
    tot=tot.append(df)
tot=tot.reset_index(drop=True)

In [ ]:
pref = pd.DataFrame({'account':['Royce Dental Surgery (Bidadari)','CITIZEN DENTAL SURGERY','Implant Dental Centre'], 
                     'pref_time': [t1,t6,t4],'pref_day':[2,4,1],'city':['d13','d23','d10']})

# pref = pd.DataFrame({'account':['Tooth Matters Dental Surgery (Toa Payoh)'], 
#                      'pref_time': [t3],'pref_day':[2],'city':['d12']}, index=[0])

pref = pref.set_index('account')

In [ ]:
pref

In [ ]:
pref=pref
df=tot
temp1 = crmplanner(pref,df)
temp1 = temp1.reset_index()
temp1.to_csv('crm_calendar_v1.csv',index=False)

In [ ]:
csv2google(temp1).to_csv('crm_calendar.csv',index=False)

In [ ]:
csv2google(temp1)